In [ ]:
# Buňka 1: Importy
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

/home/luk-hrd/miniconda3/envs/mistral-lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Knihovny úspěšně naimportovány.


In [ ]:
# Buňka 2: Konfigurace
# Použijeme přímo identifikátor modelu z Hugging Face Hub
base_model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# --- Konfigurace kvantizace (zůstává stejná, je nutná pro běh na vaší GPU) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"Bude se načítat originální model: {base_model_name}")

✅ Konfigurace připravena.
Bude se načítat originální model: mistralai/Mistral-7B-Instruct-v0.1


In [ ]:
try:
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    print("Tokenizer načten.")

    # Explicitně řekneme, aby se pro GPU s ID 0 použilo maximum paměti
    # a pro CPU se nepoužívala žádná paměť na offloadování vrstev.
    max_memory_map = {0: "8GiB", "cpu": "0GiB"} 

    # Načtení kvantizovaného modelu s explicitním řízením paměti
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map="auto",
        max_memory=max_memory_map, # Přidán osvědčený parametr
        torch_dtype=torch.bfloat16,
    )
    
    # Důležité: Přepneme model do evaluačního módu
    model.eval()
    
    print("\n Originální model je načten a připraven k chatu!")

except Exception as e:
    import traceback
    print("\n CHYBA PŘI NAČÍTÁNÍ MODELU!")
    traceback.print_exc()

Tokenizer načten.


Loading checkpoint shards: 100%|██████████| 2/2 [00:27<00:00, 13.56s/it]



 Originální model je načten a připraven k chatu!


In [4]:
chat_history = []

print("Chat s ORIGINÁLNÍM modelem je připraven! Můžete začít psát.")
print("Napište 'konec' nebo 'exit' pro ukončení chatu.")
print("-" * 50)

while True:
    user_input = input("Vy: ")
    if user_input.lower() in ["konec", "exit", "quit"]:
        print("Chat ukončen.")
        break
        
    # 2. Sestavení promptu s historií (formát pro Mistral-Instruct)
    history_prompt_parts = []
    for user_msg, assistant_msg in chat_history:
        history_prompt_parts.append(f"[INST] {user_msg} [/INST] {assistant_msg}</s>")
    history_prompt = "".join(history_prompt_parts)
    
    final_prompt = f"<s>{history_prompt}[INST] {user_input} [/INST]"
    
    # 3. Tokenizace a příprava vstupu
    inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)
    input_token_len = inputs.input_ids.shape[1]
    
    print("\nAsistent přemýšlí...")
    
    # 4. Generování odpovědi
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # 5. Dekódování a vyčištění odpovědi
    generated_tokens = outputs[0, input_token_len:]
    clean_response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    print(f"\nAI Asistent (Originál): {clean_response}")
    print("-" * 50)
    
    # 6. Aktualizace historie konverzace
    chat_history.append((user_input, clean_response))

Chat s ORIGINÁLNÍM modelem je připraven! Můžete začít psát.
Napište 'konec' nebo 'exit' pro ukončení chatu.
--------------------------------------------------

Asistent přemýšlí...

AI Asistent (Originál): A quantum state is a mathematical description of the physical properties of a quantum system at a particular time. It includes information about the probability distribution of different possible outcomes when measurements are made on the system. In other words, it represents the current state of the system in terms of its potential future behavior. Quantum states can be represented using mathematical equations such as Schrödinger's equation and can be visualized using diagrams like density matrices or wave functions.
--------------------------------------------------

Asistent přemýšlí...

AI Asistent (Originál): A quantum state is a mathematical description of the physical properties of a quantum system at a particular time. It includes information about the probability distributio